In [2]:
global file
file = input('Selet a folder') ##enter your file path here - the file is in the repo "summary_stats.csv".
##file

In [3]:
from datetime import datetime
from datetime import date
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import math
import pandas as pd

In [4]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

Get the list of spaces. We don't know how many spaces we have, so we have to cycle and only stop when the list is not filled with 1000 rows

In [5]:
spaces=[]
spaces = snapshot.Query.spaces(
  orderBy='created',
  orderDirection='asc',
  first=1000,
)
spaces_list = sg.query_df([
    spaces.id
    ]
  )


In [6]:
spaces2 = []
n=0
t=2
while n<1:
  spaces2 = snapshot.Query.spaces(
    orderBy='created',
    orderDirection='asc',
    first=1000,
    skip = 1000*t
  )
  spaces_list2 = sg.query_df([
    spaces2.id
    ]
  )
  spaces_list = pd.concat([spaces_list,spaces_list2])
  t=t+1
  l=len(spaces_list2)
  print(t,l)
  if len(spaces_list2)<1000: n=1

##olympus_coord=spaces_list.index[spaces_list['spaces_id'] == 'olympusdao.eth'].tolist()


3 1000
4 1000
5 1000
6 1000
7 1000
8 1000
9 1000
10 310


In [7]:
numspaces = len(spaces_list)
numspaces

8310

We get the list of elections  for each space. Note the choice of vote only carries the ID not the question

In [8]:
space_name = spaces_list.iat[1,0]
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='asc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots = sg.query_df([
    proposals.title,
    proposals.id,
    proposals.body,
    proposals.scores,
    proposals.scores_total
])

proposals_snapshots.insert(0,'dao',space_name)

proposals_snapshots

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
1,fabien.eth,Test,QmbGQCSWXqJwUUksSJ5UbYgjnrpn25aJNyuLFUA48wYg8Q,**Bold test** Lost of text\n\n# Proposal: Test...,0.000000,0.000000
2,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,0.000000,12.503688
3,fabien.eth,Test proposal,QmPDvn4oamr1ZxdTXH3pCKn4jpxTJUv3yMVrCBxfgDmyEA,This is a test,12.503688,12.503688
4,fabien.eth,Test proposal,QmWqUBX1jgWzs8A8hMAV2bSk8jm2HQ3wdq4FWgt4QR5U15,This is a test,18.000000,29.000000
...,...,...,...,...,...,...
870,fabien.eth,Test webhook,0xa66466451daeca21e2d50188d3b72ae6cd36626ca44f...,,52.784172,55.784172
871,fabien.eth,Test webhook,0xa66466451daeca21e2d50188d3b72ae6cd36626ca44f...,,1.000000,55.784172
872,fabien.eth,Test for EIP-1271 on Optimism,0xe0fd8430e73de3b4790648b6edb9bba0b3255c3fe7fa...,,0.000000,4445.481307
873,fabien.eth,Test for EIP-1271 on Optimism,0xe0fd8430e73de3b4790648b6edb9bba0b3255c3fe7fa...,,2018.027507,4445.481307


In [9]:
space_name = spaces_list.iat[2,0]

proposals2 = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=1000,
  where=[
    snapshot.Proposal.space == space_name,
    snapshot.Proposal.state == 'closed'
    ##snapshot.Proposal.title == 'OIP-18: Reward rate framework and reduction',
  ]
)

proposals_snapshots2 = sg.query_df([
    proposals2.title,
    proposals2.id,
    proposals2.body,
    proposals2.scores,
    proposals2.scores_total
])

proposals_snapshots2.insert(0,'dao',space_name)

proposals_snapshots2

,dao,proposals_title,proposals_id,proposals_body,proposals_scores,proposals_scores_total
0,ichi.eth,Volatility Increase in Vault IRR Deposits,0xb6acfcda2b1c19a231f7df2c6cd7bcd5654a8be989e2...,Summary:\nPropose an increase in vault rewards...,43073.307504,43073.307504
1,ichi.eth,Volatility Increase in Vault IRR Deposits,0xb6acfcda2b1c19a231f7df2c6cd7bcd5654a8be989e2...,Summary:\nPropose an increase in vault rewards...,0.000000,43073.307504
2,ichi.eth,ICHI Proposal for Bancor V3,0xefd26a12611d809411277a23ab59d41c9b56703a2faa...,"Summary:\n-Provide 50,000 BNT worth of Legacy ...",43073.307504,43073.307504
3,ichi.eth,ICHI Proposal for Bancor V3,0xefd26a12611d809411277a23ab59d41c9b56703a2faa...,"Summary:\n-Provide 50,000 BNT worth of Legacy ...",0.000000,43073.307504
4,ichi.eth,ICHI Vaults Proposal,0x9e724ac779d020df40016be159a4a8062965a84fee63...,Summary:\nProposal allocating $ICHI to the fol...,94158.348758,94210.981940
...,...,...,...,...,...,...
105,ichi.eth,Should we launch a community website on ichi.org?,Qmbj2pu23vbNPJGSMt1bgJjK4YGxcDMjeqggyLNHgDbG5V,Proposal from samwisegardener in the ichi disc...,0.000000,109450.440901
106,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,9039.392908,9039.392908
107,ichi.eth,test,QmPmUra8QTkymdJNKT2QjYUtJtXpVY6wE4QT66genw9DQZ,test,0.000000,9039.392908
108,ichi.eth,Test?,QmRRJ8uwthPoTCmGxk2TANdhDnppECvKBcaz55FywySRP4,Test - yes or no,0.000000,0.000000


In [10]:
proposals3 = []
t=int(input('set starting point for DAOs'))
limit=int(input('set limit for DAOs to pull (-1 for all records)'))+2
breaker=int(input('set size of chunk - smaller for best perf (100?)'))
if limit==-1:
  limit = numspaces

while t<=limit: ##numspaces when out of testing
  space_name = spaces_list.iat[t,0]
  proposals3 = snapshot.Query.proposals(
    orderBy='created',
    orderDirection='desc',
    first=1000,
    skip = 1000*t,
    where=[
      snapshot.Proposal.space == space_name,
      snapshot.Proposal.state == 'closed'
      ]
  )
  proposals_snapshots3 = sg.query_df([
    proposals3.title,
    proposals3.id,
    proposals3.body,
    proposals3.scores,
    proposals3.scores_total
])
  proposals_snapshots3.insert(0,'dao',space_name)
  proposals_snapshots = pd.concat([proposals_snapshots,proposals_snapshots2,proposals_snapshots3])
  t=t+1
  l=len(proposals_snapshots3)
  if t%10 == 0: print (t,space_name,l)
  if t>1150: print (t,space_name,l)
  if len(proposals_snapshots3)>0:
    print(t,space_name,l)

proposals_snapshots

ValueError: invalid literal for int() with base 10: ''

In [ ]:
total_props = (int(len(proposals_snapshots)))-1
chunks = math.ceil(total_props/breaker)
print(total_props,'elections. Pulling in', chunks,'sets of',breaker,'records.' )

We are going to use this to timestamp the filenames

In [ ]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
  )


In [ ]:
##spit out the file!
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(proposals_snapshots))+'_snapshots.csv'
proposals_snapshots.to_csv(path, index = False)
print('file saved: '+ path)

In [ ]:
proposal_id = proposals_snapshots.iloc[1,2]

vote_tracker = snapshot.Query.votes(
orderBy = 'created',
orderDirection='desc',
first=10000,
where=[
  snapshot.Vote.proposal == proposal_id
]
)


voting_snapshots_list = sg.query_df([
  vote_tracker.id,
  vote_tracker.voter,
  vote_tracker.created,
  vote_tracker.choice,
  vote_tracker.vp
])
print(proposal_id)
voting_snapshots_list

In [ ]:
voting_snapshots_list['Proposal'] = proposal_id
print(voting_snapshots_list)

In [ ]:
n=2
iteration =1
print('pull started')
endpoint = breaker
while iteration <= chunks:
  while n<=min(endpoint*iteration,total_props): ##total_props:
    proposal_id = proposals_snapshots.iloc[n,2]
    ##if n%10 == 0: print (n,proposal_id)
    n=n+1
    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=1000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ])
    voting_snapshots_list2 = sg.query_df([
      vote_tracker.id,
      vote_tracker.voter,
      vote_tracker.created,
      vote_tracker.choice,
      vote_tracker.vp
    ])
    voting_snapshots_list2['Proposal'] = proposal_id
    voting_snapshots_list = pd.concat([voting_snapshots_list,voting_snapshots_list2])
    iteration = iteration +1
    iter_progress = 100*round(n/5000,4)
    remaining = (total_props-n)
    if n%500==0:
      print(str(n), 'proposals pulled.',str(remaining), 'remain.','\nProgress in this chunk:', str(iter_progress),'%')

      ##now we save that part in its file
  path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals_part_'+str(iteration)+'.csv'
  voting_snapshots_list.to_csv(path, index = False)
  chunk_progress = 100*round(remaining/total_props,2)
  print('file saved:',path, str(n),'proposals pulled',
        '\nprogress in whole download', str(chunk_progress),'%')




And finally, the body of each election's choices. Not critical but useful to have.

Still needs work! Having a tough time automating pulling all those choices into a single table.

In [ ]:
proposals_choices = sg.query(proposals.choices)
proposals_choices

In [ ]:
numchoices = len(pd.DataFrame(proposals_choices).columns)
numchoices

In [ ]:
##placing 25 options just in case. Could be more, but I hope not
proposals_choices = pd.DataFrame(proposals_choices)

##proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_1', 'option_2', 'option_3', 'option_4', 'option_5','option_6','option_7','option_8','option_9','option_10','option_11','option_12','option_13','option_14','option_15','option_16','option_17','option_18','option_19','option_20','option_21','option_22','option_24','option_24','option_25]'])

In [ ]:
dao_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [ ]:
dao_governance_view

In [ ]:
##spit out the file!
path =file+'/votes_table_'+str(date.today().strftime("%b-%d-%Y"))+'_'+str(len(voting_snapshots_list))+'_proposals.csv'
voting_snapshots_list.to_csv(path, index = False)
print('file saved')